In [1]:
import syft as sy
sy.requires(">=0.8,<0.8.1")

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/yunhs


✅ The installed version of syft==0.8.0 matches the requirement >=0.8 and the requirement <0.8.1


In [17]:
node = sy.orchestra.launch(name="test-domain-1", port=8080, dev_mode=True)

Starting test-domain-1 server on 0.0.0.0:8080

SQLite Store Path:
!open file:///var/folders/4s/vsgdjz495453yymsd09nx80c0000gn/T/7bca415d13ed1ec841f0d0aede098dbb.sqlite

> Domain: test-domain-1 - 7bca415d13ed1ec841f0d0aede098dbb - NodeType.DOMAIN

Services:
ActionService
DataSubjectMemberService
DataSubjectService
DatasetService
MessageService
MetadataService
NetworkService
PolicyService
ProjectService
RequestService
UserCodeService
UserService


INFO:     Started server process [3556]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)


INFO:     127.0.0.1:50199 - "GET /api/v1/new/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:50199 - "POST /api/v1/new/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:50199 - "GET /api/v1/new/api?verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f HTTP/1.1" 200 OK
INFO:     127.0.0.1:50204 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:50207 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:50210 - "GET /api/v1/new/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:50213 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:50216 - "GET /api/v1/new/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:50219 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:50222 - "GET /api/v1/new/api?verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f HTTP/1.1" 200 OK
INFO:     127.0.0.1:50225 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:50228 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
IN

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [3556]


In [18]:
domain_client = node.login(email="info@openmined.org", password="changethis")

In [19]:
from typing import List, Dict, Any, Optional

In [20]:
class RepeatedCallPolicy(sy.CustomOutputPolicy):
    n_calls: int = 0
    downloadable_output_args: List[str] = []
    state: Dict[Any, Any] = {}

    def __init__(self, n_calls=1, downloadable_output_args: List[str] = None):
        self.downloadable_output_args = downloadable_output_args if downloadable_output_args is not None else []
        self.n_calls = n_calls + 1
        self.state = {"counts": 0}

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, context, outputs):
        output_dict = {}
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                output_dict[output_arg] = outputs[output_arg]

            self.state["counts"] += 1
        else:
            return None

        return output_dict

In [21]:
policy = RepeatedCallPolicy(n_calls=1, downloadable_output_args=['y'])

In [22]:
policy.n_calls

2

In [23]:
policy.downloadable_output_args

['y']

In [24]:
policy.init_kwargs

{'n_calls': 1, 'downloadable_output_args': ['y']}

In [25]:
print(policy.init_kwargs)
a_obj = sy.ActionObject.from_obj({'y': [1,2,3]})
x = policy.apply_output(None, a_obj)
x

{'n_calls': 1, 'downloadable_output_args': ['y']}


,key,type,id
0,y,"[1, 2, 3]",a41ef16f539f4190bd83d5b9c114b33f


In [26]:
policy.n_calls

2

In [27]:
import numpy as np
x = np.array([1,2,3])
x_pointer = sy.ActionObject.from_obj(x)
domain_client.api.services.action.save(x_pointer)

<class 'syft.service.response.SyftSuccess'>: <class 'syft.service.action.numpy.NumpyArrayObject'> saved

In [28]:
@sy.syft_function(
    input_policy=sy.ExactMatch(x=x_pointer),
    output_policy=RepeatedCallPolicy(n_calls=10, downloadable_output_args=['y']),
)
def func(x):
    return {"y": x+1}

In [29]:
@sy.syft_function(input_policy=sy.ExactMatch(x=x_pointer),
                  output_policy=sy.SingleExecutionExactOutput())
def train_mlp(x):
    return x

In [30]:
domain_client.api.services.code.request_code_execution(func)

```python
class Request:
  id: str = 639482d6b7c14d3195f002386dfe5813
  requesting_user_verify_key: str = aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f
  approving_user_verify_key: str = None
  request_time: str = 2023-05-01 23:40:05
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  request_hash: str = "01dcad7a4bc805bb5fbe45db806e6e5030b5e7109b91345ca1d73cb85a8b0772"
  changes: str = [syft.service.request.request.UserCodeStatusChange]

```

In [31]:
from syft import MessageStatus
messages = domain_client.api.services.messages.get_all_for_status(MessageStatus.UNDELIVERED)
messages

,type,id,subject,status,created_at,linked_obj
0,syft.service.message.messages.Message,2bd8d96724bc48a587d3a6debf86b31a,Approval Request,MessageStatus.UNDELIVERED,2023-05-01 23:36:24,<<class 'syft.service.request.request.Request'...
1,syft.service.message.messages.Message,a8f3c6f659ea42449b03ad9b874efca2,Approval Request,MessageStatus.UNDELIVERED,2023-05-01 23:40:05,<<class 'syft.service.request.request.Request'...
2,syft.service.message.messages.Message,8cf4d5be61324b7eadc6e22871326461,Project Approval,MessageStatus.UNDELIVERED,2023-05-01 23:22:20,<<class 'syft.service.project.project.Project'...
3,syft.service.message.messages.Message,1609c587f6b34bf4b193785a264772f2,Approval Request,MessageStatus.UNDELIVERED,2023-05-01 23:27:26,<<class 'syft.service.request.request.Request'...
4,syft.service.message.messages.Message,fcb593575fa340f4a65be8c4357015db,Approval Request,MessageStatus.UNDELIVERED,2023-05-01 23:21:05,<<class 'syft.service.request.request.Request'...


In [32]:
from syft.service.request.request import ObjectMutation, UserCodeStatusChange, Request
func = None
request = None
for message in messages:
    req = message.linked_obj.resolve
    if isinstance(req, Request):
        for change in req.changes:
            if isinstance(change, UserCodeStatusChange):
                user_code = change.linked_obj.resolve
                if "func" in user_code.service_func_name:
                    func = user_code
                    request = req

In [33]:
func

```python
class UserCode:
  id: str = 6245f3ae63564f388bf6327a2cfac0cd
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  user_verify_key: str = aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f
  raw_code: str = "@sy.syft_function(
    input_policy=sy.ExactMatch(x=x_pointer),
    output_policy=RepeatedCallPolicy(n_calls=10, downloadable_output_args=['y']),
)
def func(x):
    return {"y": x+1}
"
  input_policy_type: str = <class 'syft.service.policy.policy.ExactMatch'>
  input_policy_init_kwargs: str = {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): {'x': <UID: cae382f2251140f39f72739df4db9fb4>}}
  input_policy_state: str = b''
  output_policy_type: str = syft.service.policy.policy.UserPolicy
  output_policy_init_kwargs: str = {'n_calls': 10, 'downloadable_output_args': ['y']}
  output_policy_state: str = b''
  parsed_code: str = "def user_func_func_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_d7f8db0378b044d89c2d33a89c7eeba45f24ae57f515a61a0dd3027bbf674862(x):

    def func(x):
        return {'y': x + 1}
    result = func(x=x)
    return result"
  service_func_name: str = "func"
  unique_func_name: str = "user_func_func_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_d7f8db0378b044d89c2d33a89c7eeba45f24ae57f515a61a0dd3027bbf674862"
  user_unique_func_name: str = "user_func_func_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f"
  code_hash: str = "d7f8db0378b044d89c2d33a89c7eeba45f24ae57f515a61a0dd3027bbf674862"
  signature: str = (x)
  status: str = {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): <UserCodeStatus.SUBMITTED: 'submitted'>}
  input_kwargs: str = ['x']
  enclave_metadata: str = None

```

In [34]:
for message in messages:
    domain_client.api.services.messages.mark_as_delivered(message.id)

In [35]:
request.approve()

<class 'syft.service.response.SyftSuccess'>: Request 639482d6b7c14d3195f002386dfe5813 changes applied

In [36]:
request.changes

,type,id
0,syft.service.request.request.UserCodeStatusChange,0201063897124acda350c348f921ce2a


In [37]:
request.changes[0]

```python
class UserCodeStatusChange:
  id: str = 0201063897124acda350c348f921ce2a
  linked_obj: str = <<class 'syft.service.code.user_code.UserCode'>: 6245f3ae63564f388bf6327a2cfac0cd@<Node: 7bca415d13ed1ec841f0d0aede098dbb>
  value: str = UserCodeStatus.EXECUTE
  match_type: str = True

```

In [38]:
result = func.unsafe_function(x=x_pointer)
result

,key,type,id
0,y,"array([2, 3, 4])",c15a0d0920494be89330b747ad4513fa


In [39]:
final_result = request.accept_by_depositing_result(result) 
final_result

<class 'syft.service.response.SyftSuccess'>: Request 639482d6b7c14d3195f002386dfe5813 changes applied

In [40]:
res = domain_client.api.services.code.func(x=x_pointer)
res

{'y': array([2, 3, 4])}

In [41]:
assert (res["y"] == np.array([2, 3, 4])).all()

In [42]:
assert set(res.syft_action_data.keys()) == set(list("y"))


In [43]:
domain_client.api.services.code.get_all()[0].output_policy

```python
class OutputPolicyExecuteOnce:
  id: str = d78c4238249044a3a402fde2841853ce
  init_kwargs: str = {}
  output_history: str = [syft.service.policy.policy.OutputHistory]
  output_kwargs: str = []
  node_uid: str = None
  count: str = 1
  limit: str = 1

```

In [44]:
domain_client.api.services.policy.get_all()

[]

In [45]:
output_policy = domain_client.api.services.policy.get_all()
output_policy

[]

In [46]:
node.land()

Stopping test-domain-1
